# Webscrapping Project : Concerts Map 

## Webscrapping concerts data from the website offi

### Imports

In [15]:
import re 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

from IPython.display import display
from geopy.geocoders import Nominatim
import folium
import time

from flask import Flask, render_template, jsonify
from IPython.core.display import display, HTML
from pyngrok import ngrok


### Extracting data 

In [2]:
url        = "https://www.offi.fr/concerts/prochainement.html"
requesting = requests.get(url)
soup       = BeautifulSoup(requesting.content, "html.parser")

In [4]:
concerts_layout=soup.find(class_='mini-fiche-container populaire-layout') # Un layout avec l'ensemble des informations sur les concerts dans des fiches

concerts =[]

for concert in concerts_layout.find_all(class_='column'):
    title = concert.find('span', itemprop='name').get_text(strip=True)
    location = concert.find(class_='event-place pt-2').get_text(strip=True)
    date = concert.find('b').get_text(strip=True)
    concerts.append({"title": title, "location": location, "date": date})

for concert in concerts:
    print (concert)

{'title': 'Boris Grebenshchikov', 'location': 'Le Trianon', 'date': 'Samedi 23 Décembre : 19h00'}
{'title': 'Boudchart', 'location': "L'Olympia", 'date': 'Lundi 1er Janvier : 16h00, 20h30, Mardi 2 Janvier : 20h30'}
{'title': 'Umberto Tozzi', 'location': "L'Olympia", 'date': 'Dimanche 7 Janvier : 20h00'}
{'title': 'The Musical Story of Elvis', 'location': "L'Olympia", 'date': 'Lundi 8 Janvier : 20h00'}
{'title': 'Kery James', 'location': 'Salle Pleyel', 'date': 'Mercredi 10 Janvier : 20h00'}
{'title': 'Louis Bertignac', 'location': 'Salle Pleyel', 'date': 'Samedi 13 Janvier : 20h00'}
{'title': 'Jay Chou', 'location': 'Paris La Défense Arena', 'date': 'Samedi 13 Janvier : 20h00'}
{'title': 'Hugues Aufray', 'location': "L'Olympia", 'date': 'Dimanche 14 Janvier : 20h00'}
{'title': 'One Night of Queen', 'location': 'Le Zénith Paris - La Villette', 'date': 'Mar 16 Janvier : 20h00'}
{'title': 'Slowdive', 'location': 'La Cigale', 'date': 'Mercredi 17 Janvier : 20h00'}
{'title': 'Simple Plan', 

### Map of concerts

In [9]:
def geocode_location(location):
    geolocator = Nominatim(user_agent="geoapiExercises")
    try:
        loc = geolocator.geocode(location)
        return (loc.latitude, loc.longitude)
    except:
        return None


# Geocode each concert location
for concert in concerts:
    coords = geocode_location(concert['location'])
    if coords:
        concert['coords'] = coords
    else:
        concert['coords'] = (0, 0)  # Default coords, adjust as necessary
    time.sleep(1)  # To avoid hitting request limits on the geocoding API

# Create a map and add markers
map = folium.Map(location=[48.8566, 2.3522], zoom_start=12)  # Centered on Paris
for concert in concerts:
    folium.Marker(
        location=concert['coords'],
        popup=f"{concert['title']} - {concert['date']}",
        tooltip=concert['title']
    ).add_to(map)

In [10]:
display(map)

### API

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html', concerts=concerts)

@app.route('/api/concerts')
def get_concerts():
    return jsonify(concerts_data)

# Start ngrok and create a tunnel to the Flask app
public_url = ngrok.connect(port=5000)

# Print the public URL
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Run the Flask app in the background
app_thread = threading.Thread(target=app.run, kwargs={'debug': False})
app_thread.start()

html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Concert Map</title>
    <!-- Include Leaflet CSS and JS -->
    <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
    <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
</head>
<body>

<div id="map" style="height: 500px;"></div>

<script>
    var map = L.map('map').setView([48.8566, 2.3522], 12); // Set the initial map view

    // Add a tile layer
    L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
        attribution: '&copy; OpenStreetMap contributors'
    }).addTo(map);

    // Fetch concerts data from the API
    fetch('{}/api/concerts'.format(window.location.origin))
        .then(response => response.json())
        .then(concerts => {
            concerts.forEach(concert => {
                // Split the date string to get individual dates
                var dates = concert.date.split(', ');

                // Create a popup content with concert information
                var popupContent = "<b>" + concert.title + "</b><br>" + concert.location + "<br>" + dates.join("<br>");

                // Create a marker for each concert and bind a popup
                L.marker([48.8566, 2.3522]) // Replace with the actual latitude and longitude of the location
                    .bindPopup(popupContent)
                    .addTo(map);
            });
        });
</script>

</body>
</html>
"""

display(HTML(html_code))
